Code to prepare and plot the altimetry input data files, both for Diva2D and [divand](https://www.github.com/gher-ulg/divand.jl/).

In [1]:
import os
import re
import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import rcParams
import netCDF4
import logging
import divaaltimetry
from calendar import monthrange

## Configuration

Logging

In [2]:
logger = logging.getLogger("diva-altimetry")
logger.setLevel(logging.DEBUG)

Plotting (should be ideally in `matplotlibrc` but seems not to work with virtualenv).

In [3]:
rcParams.update({'font.size': 16, 'figure.dpi': 300, 'savefig.bbox': 'tight'})

# Input parameters

## Files and directories

In [44]:
# databasedir = "/home/ctroupin/Data/AVISO/regional-mediterranean/delayed-time/along-track/filtered/adt"
databasedir = "/home/ctroupin/Data/AVISO/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049"
divaoutputdir = "../data_cmems/diva/"
divandoutputdir = "../data_cmems/divand/"
figdir = "../figures/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/"
missionlist = ["al", "c2", "h2", "j2"]
missiondirlist = ["dataset-duacs-rep-medsea-al-phy-unfiltered-l3-v3",
                  "dataset-duacs-rep-medsea-c2-phy-unfiltered-l3-v3",
                  "dataset-duacs-rep-medsea-h2-phy-unfiltered-l3-v3",
                  "dataset-duacs-rep-medsea-j2-phy-unfiltered-l3-v3"]

if not os.path.exists(divaoutputdir):
    os.makedirs(divaoutputdir)
if not os.path.exists(divandoutputdir):
    os.makedirs(divandoutputdir)
if not os.path.exists(figdir):
    os.makedirs(figdir)

## Domain and period of interest
* coordinates: tuple with lon_min, lon_max, lat_min, lat_max 
* timescale: temporal scale in days; used to compute weights for Diva2D

In [50]:
coordinates = (-6.75, 40.001, 30, 48.)
meridians = np.arange(-8., 40., 8.)
parallels = np.arange(30., 50., 4.5)
yearlist, monthlist, daylist, interval = [2014], range(1, 12), range(1, 31), 10
timescale = 5.        

## Map projection
Define the projection from the coordinates; used for the plots.

In [51]:
m = Basemap(projection='merc',
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

## Generate list of files for the selected period

Based on the filenames, we can easily identify the files corresponding to the time of interest:
* datemid is the central time of the considered period
* datestart is the start date
* dateend is the last day.

# Main loop
Goal: create a list of files that will be put together to create a daily data file that will serve as an input for `diva` and `divand`.    
Loop on years, months and days; for each day, create a list of the files that contains the dates of interest (day +/- the selected interval).

In [55]:
for year in set(yearlist):
    
    for month in set(monthlist):
        daystart = 1 
        dayend = monthrange(year, month)[1]
        if month == monthlist[0]:
            daystart = daylist[0]
        if month == monthlist[-1]:
            dayend = daylist[-1]
        
        for day in range(daystart, dayend + 1):
            logger.debug("year: {0}, month: {1}, day: {2}".format(year, month, day))
            
            # Create the file list:
            # find files with a date around the data of interest +/- N days
            filelist = divaaltimetry.make_filelist(databasedir, missionlist, missiondirlist,
                                                   year, month, day, interval)
            
            logger.debug("{0} file(s) in filelist".format(len(filelist)))
            datemid = datetime.datetime(year, month, day).toordinal() - \
            datetime.datetime(1950, 1, 1).toordinal()
            logging.info("Central time (in days since January 1st, 1950): {0}".
                         format(datemid))

            fignamesuffix, figtitledate = divaaltimetry.prepare_datestrings(year, 
                                                                            month, 
                                                                            day, interval)
            logger.info(fignamesuffix)

            # File and figure names
            # Generate file and figure names using the suffixes.
            divandfile = "SLA_{0}.dat".format(fignamesuffix)
            divafile =  "data_{0}.dat".format(fignamesuffix)
            figname = "SLA_{0}".format(fignamesuffix)
            logger.info("Text file name: {0}".format(outputfilename))
            logger.info("Figure name: {0}".format(figname))

            # Make the plot and write data to text files
            
            divaaltimetry.plot_data_tracklist(filelist, m, os.path.join(figdir, figname), figtitledate, 
                                              meridians, parallels,
                                              vmin=-0.20, vmax=0.20, cmap=plt.cm.RdYlBu_r)
            
            divaaltimetry.write_tracks2file(filelist, 
                                            os.path.join(divandoutputdir, divandfile), 
                                            os.path.join(divaoutputdir, divafile), 
                                            timescale, datemid)
            
logger.info("Summary:")
logger.info("Figures available in {}".format(os.path.abspath(figdir)))
logger.info("Files for diva (2D) available in {}".format(os.path.abspath(divaoutputdir)))
logger.info("Files for divand available in {}".format(os.path.abspath(divandoutputdir)))

DEBUG:diva-altimetry:year: 2014, month: 1, day: 1
DEBUG:diva-altimetry:30 file(s) in filelist
INFO:diva-altimetry:20131222_20140111
INFO:diva-altimetry:Text file name: SLA_20131222_20140111.dat
INFO:diva-altimetry:Figure name: SLA_20131222_20140111
DEBUG:diva-altimetry:year: 2014, month: 1, day: 2
DEBUG:diva-altimetry:33 file(s) in filelist
INFO:diva-altimetry:20131223_20140112
INFO:diva-altimetry:Text file name: SLA_20131222_20140111.dat
INFO:diva-altimetry:Figure name: SLA_20131223_20140112


KeyboardInterrupt: 